# Loading single-end data from SRA into QIIME 2 2019.1. 

This example illustrates loading of the files obtained from SRA [BioProject PRJNA516932](https://www.ncbi.nlm.nih.gov/bioproject/516932). 


The two files that need to be obtained from SRA directly are the accession list (`SRR_Acc_List.txt`) and the sample metadata or "run table" (`SraRunTable.txt`). The sample metadata can be loaded with QIIME 2 after the `Sample name` column is moved to be the first column, and renamed to be one of the allowed QIIME 2 sample id column names.

```bash
$ conda create -n sra-tools -c bioconda sra-tools
$ conda activate sra-tools

$ head SRR_Acc_List.txt
SRR8484238
SRR8484239
SRR8484240
SRR8484241
SRR8484242
SRR8484243
SRR8484244
SRR8484245
SRR8484246
SRR8484247

$ prefetch --option-file SRR_Acc_List.txt -O sra

$ fastq-dump –split-files --gzip sra/*sra -O fastq

```

In [10]:
from qiime2 import Metadata
md = Metadata.load('./SraRunTable.txt')
f = open('fastq-manifest.csv', 'w')
f.write('sample-id,absolute-filepath,direction\n')
for sample_id, base_filename in md.get_column('Run').to_series().items():
    f.write('%s,$PWD/fastq/%s.fastq.gz,forward\n' % (sample_id, base_filename))
f.close()

In [11]:
!head fastq-manifest.csv

sample-id,absolute-filepath,direction
udca.trial.29TPOBOH855UL,$PWD/fastq/SRR8484238.fastq.gz,forward
udca.trial.2S9ZE5I92VCMH,$PWD/fastq/SRR8484239.fastq.gz,forward
udca.trial.6YUR40U7Y49B6,$PWD/fastq/SRR8484240.fastq.gz,forward
udca.trial.3V55PNQITVUFE,$PWD/fastq/SRR8484241.fastq.gz,forward
udca.trial.2WI4ZN13D48J0,$PWD/fastq/SRR8484242.fastq.gz,forward
udca.trial.5O4AHMMLWBNCF,$PWD/fastq/SRR8484243.fastq.gz,forward
udca.trial.VGV3YP7FM8RG,$PWD/fastq/SRR8484244.fastq.gz,forward
udca.trial.VWVP688Q64JI,$PWD/fastq/SRR8484245.fastq.gz,forward
udca.trial.2J11QU8XEKXCL,$PWD/fastq/SRR8484246.fastq.gz,forward


In [17]:
from qiime2 import Artifact

a = Artifact.import_data('SampleData[SequencesWithQuality]', 'fastq-manifest.csv', view_type='SingleEndFastqManifestPhred33')
a.save('demux.qza')


'demux.qza'

In [18]:
import qiime2.plugins.demux

In [23]:
v = qiime2.plugins.demux.actions.summarize(a).visualization
v.save('demux.qzv')

'demux.qzv'